Dê ao google Colab a autorização para acessar seu google drive a partir do comando abaixo:

In [ ]:
!python -m pip install pyyaml==5.1

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.structures import BoxMode
from detectron2.data import transforms as T
from google.colab import drive
drive.mount('/content/drive')


def get_pod_dicts(img_dir):
    img_dir = os.path.join('/content/drive/MyDrive/' + img_dir)

    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}

        augs = T.AugmentationList([
            T.RandomBrightness(0.9, 1.1),
            T.RandomFlip(prob=0.5),
            T.RandomCrop("absolute", (640, 640))
        ])
        
        filename = os.path.join(img_dir, v["filename"])

        input = T.AugInput(cv2.imread(filename), boxes=boxes, sem_seg=sem_seg)
        transform = augs(input)

        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for anno in annos:
            assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

#Comment the following lines if not running for the first time
for d in ["train", "val"]:
    DatasetCatalog.register("pod_" + d, lambda d=d: get_pod_dicts("pod/" + d))
    MetadataCatalog.get("pod_" + d).set(thing_classes=["pod"])
pod_metadata = MetadataCatalog.get("pod_train")

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("pod_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (pod).
if(!os.path.join("/content/drive/MyDrive/model_final.pth"))):
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
if(os.path.join("/content/drive/MyDrive/model_final.pth"))):
    cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_final.pth"
else:
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
    predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_pod_dicts("pod/val")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=pod_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("pod_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "pod_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

In [ ]:
from numpy.ma.core import mean
import math
import torch
soybean_csv = pd.read_excel('/content/drive/MyDrive/SOJA_v00.xlsx', sheet_name='SOJA_v00')
values = soybean_csv.values.tolist()
podPerSoy = {}

for line in values:
  seedsInSoy = 0
  podPerSoy[line[2]] = line[4]
  im = cv2.imread(f"/content/drive/MyDrive/images/{line[2]}.jpg")

  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
    metadata=pod_metadata,
    scale=1,
  )
  masks = outputs['instances'].pred_masks

  t = (torch.sum(torch.flatten(masks, start_dim=1),dim=1))

  for seed in (t.detach().cpu().numpy()):
    if seed < 50:
      seedsInSoy = seedsInSoy + 1
    if seed > 50 and seed < 100:
      seedsInSoy = seedsInSoy + 2
    if seed > 100 and seed < 150:
      seedsInSoy = seedsInSoy + 3
    if seed > 150:
      seedsInSoy = seedsInSoy + 4
  print(seedsInSoy)

  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  print(line[5])
  print(line[2])
  if(math.isnan(podPerSoy.get(line[2]))):
    break
